In [1]:
import urllib
from bs4 import BeautifulSoup
import mechanize
import redis
headers = [
    ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'),
    ('User-Agent', 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.17 (KHTML, like Gecko)  Chrome/24.0.1312.57 Safari/537.17'),
    ('Content-Type', 'application/x-www-form-urlencoded'),
    ('Referer', 'https://ssl.forumedia.eu/beach38-courtbuchung.de/index.php'),
    ('Accept-Encoding', 'gzip,deflate,sdch'),
    ('Accept-Language', 'en-US,en;q=0.8'),
]

In [11]:
help(r.lrem)

Help on method lrem in module redis.client:

lrem(self, name, count, value) method of redis.client.StrictRedis instance
    Remove the first ``count`` occurrences of elements equal to ``value``
    from the list stored at ``name``.
    
    The count argument influences the operation in the following ways:
        count > 0: Remove elements equal to value moving from head to tail.
        count < 0: Remove elements equal to value moving from tail to head.
        count = 0: Remove all elements equal to value.



In [ ]:
date = '2016-09-17'

#url = 'https://ssl.forumedia.eu/beach38-courtbuchung.de/reservations_week.php?action=showReservations&type_id=1&date={:}&page=1'.format(date)
url = 'https://ssl.forumedia.eu/beach38-courtbuchung.de/reservations.php?action=showReservations&type_id=1&date={:}&page=1'.format(date)
print url

response = mechanize.urlopen(url)
#print response.read()

soup = BeautifulSoup(response.read(), "lxml")

In [ ]:
courts =  soup.find_all(class_= "areaPeriods")

free_court = {}
order = [2,3,4,0,1]
for i in order:
    c = courts[i]
    th = c.find_all('th')[0].get_text()
    if len(th) > 0:
        if 'Soccer' not in th and 'Platz' in th: 
            avail_times = c.find_all(class_="avaliable")
            if len(avail_times) > 0:
                court = th[0:7].split(' ')[1]
                for tm in avail_times:
                    start_time, _ = tm.find('a').get_text().replace(' ', '').replace('Uhr', '').split('-')
                    free_spot = {'date': date, 'start_time': '{:}'.format(start_time), 'court': '{:}'.format(court)}

In [ ]:
r = redis.StrictRedis(host='localhost', port=6379, db=0)                                            

opt = {'date': '2016-10-22', 'start_time': '17:00', 'court':'5'}
print "https://ssl.forumedia.eu/beach38-courtbuchung.de/index.php"
print "https://ssl.forumedia.eu/beach38-courtbuchung.de/reservation_order.php?area_id={court}&date={date}&time={start_time}&page=1".format(**opt)


In [ ]:
r.lpush('2016-09-15', {'email': 'benhoyle1212@gmail.com', 'start_time': '17:00'})
r.lpush('2016-09-17', {'email': 'benhoyle1212@gmail.com', 'start_time': '15:00'})
r.lpush('2017-10-15', {'email': 'benhoyle1212@gmail.com', 'start_time': '19:00'})

In [6]:
def get_free_days(date):
    #url = 'https://ssl.forumedia.eu/beach38-courtbuchung.de/reservations_week.php?action=showReservations&type_id=1&date={:}&page=1'.format(date)
    url = 'https://ssl.forumedia.eu/beach38-courtbuchung.de/reservations.php?action=showReservations&type_id=1&date={:}&page=1'.format(date)
    response = mechanize.urlopen(url)
    soup = BeautifulSoup(response.read(), "lxml")
    courts =  soup.find_all(class_= "areaPeriods")
    free_courts = []
    order = [2,3,4,0,1]
    for i in order:
        c = courts[i]
        th_ = c.find_all('th')
        if len(th_) > 0:
            th = th_[0].get_text()
            if 'Soccer' not in th and 'Platz' in th: 
                avail_times = c.find_all(class_="avaliable")
                if len(avail_times) > 0:
                    court = th[0:7].split(' ')[1]
                    for tm in avail_times:
                        start_time, _ = tm.find('a').get_text().replace(' ', '').replace('Uhr', '').split('-')
                        free_spot = {'date': date, 'start_time': '{:}'.format(start_time), 'court': '{:}'.format(court)}
                        free_courts.append(free_spot)
    return free_courts

In [7]:
def send_email(opts, date, link):
    # Import smtplib for the actual sending function
    import smtplib

    # Import the email modules we'll need
    from email.mime.text import MIMEText

    # Open a plain text file for reading.  For this example, assume that
    # the text file contains only ASCII characters.
    email_body = "Dear vballer, \n A court is ready to book. " + \
    "\n go here {:}\nBe quick!\n".format(link) + \
    " If you are not logged in, you will need to log in and then return to this email and click again.\n" + \
    "Enjoy! \nYour friendly beach-38 robot."
    # Create a text/plain message
    print email_body
    if False:
        msg = MIMEText(email_body)

        # me == the sender's email address
        # you == the recipient's email address
        msg['Subject'] = 'Beach38 Court available on {:} at {:}'.format(date, opts['start_time'])
        msg['From'] = 'beach38_booker@no-reply.com'
        msg['To'] = opts['email']

        # Send the message via our own SMTP server, but don't include the
        # envelope header.
        s = smtplib.SMTP('localhost')
        s.sendmail(msg['From'], [opts['email']], msg.as_string())
        s.quit()

In [14]:
keys = r.keys('*')
for date in keys:
    ln = r.llen(date)
    available = get_free_days(date)
    for i_ in r.lrange(date, 0, ln):
        i = eval(i_)
        for opt in available:
            if i['start_time'] == opt['start_time']:
                link = "https://ssl.forumedia.eu/beach38-courtbuchung.de/reservation_order.php?area_id={court}&date={date}&time={start_time}&page=1".format(**opt)
                link = "<a href='{0}'>{0}</a>".format(link)
                send_email(i, date, link)
                #remove this element
                r.lrem(date, 0,  i_)
